In [18]:
from scienceworld import ScienceWorldEnv
from sources.agent import BDIAgent
from sources.scienceworld import parse_observation

from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [19]:
#hg_model_hub_name = "alisawuffles/roberta-large-wanli"
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
nli_model = NLIModel(hg_model_hub_name)

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# MAIN GOAL
green_plan = """
    IF your goal is to use chemistry to create green paint THEN
        move to art studio,
        pour cup containing blue paint in art studio into jug,
        pour cup containing yellow paint in art studio into jug,
        mix jug,
        focus on green paint
"""

orange_plan = """
    IF your goal is to use chemistry to create orange paint THEN
        move to art studio,
        pour cup containing red paint in art studio into jug,
        pour cup containing yellow paint in art studio into jug,
        mix jug,
        focus on orange paint
"""

violet_plan = """
    IF your goal is to use chemistry to create violet paint THEN
        move to art studio,
        pour cup containing red paint in art studio into jug,
        pour cup containing blue paint in art studio into jug,
        mix jug,
        focus on violet paint
"""

violet_red_plan = """
    IF your goal is to use chemistry to create violet-red paint CONSIDERING you are in the art studio THEN
        pour cup containing red paint in art studio into cup containing nothing,
        pour cup containing blue paint in art studio into cup containing red paint in table,
        mix cup containing blue paint and red paint,
        focus on paint in cup containing violet paint,
        pour cup containing violet paint into cup containing violet paint,
        open cupboard,
        pour cup containing red paint into cup containing violet paint,
        mix cup containing red paint and violet paint,
        focus on paint in cup containing violet-red paint
"""

violet_red_plan_2 = """
    IF your goal is to use chemistry to create violet-red paint CONSIDERING you are not in art studio THEN
        move to art studio,
        pour cup containing red paint in art studio into cup containing nothing,
        pour cup containing blue paint in art studio into cup containing red paint in table,
        mix cup containing blue paint and red paint,
        focus on paint in cup containing violet paint,
        pour cup containing violet paint into cup containing violet paint,
        open cupboard,
        pour cup containing red paint into cup containing violet paint,
        mix cup containing red paint and violet paint,
        focus on paint in cup containing violet-red paint
"""

all_plans = [violet_red_plan, violet_red_plan_2]

pl = PlanLibrary()
pl.load_plans_from_strings(all_plans)  # load plans from strings above
pl.load_plans_from_file("plans.txt")  # load plans from file
print(pl.plans.keys())

dict_keys(['use chemistry to create violet-red paint', 'move to art studio', 'move to bedroom', 'move to greenhouse', 'move to kitchen', 'move to living room', 'move to workshop', 'move to outside', 'move to foundry', 'move to bathroom', 'move to hallway'])


In [21]:
env = ScienceWorldEnv("", "", envStepLimit=100)

#root_event = 'use chemistry to create green paint'

task = 'chemistry-mix-paint-tertiary-color'
env.load(task, 0)
randVariationIdx = env.getRandomVariationTest()
env.load(task, 28)

goal = env.getTaskDescription().split('.')[0].replace("Your task is to", "").strip()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=goal, valid_actions=info['valid'])

goal

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Task Name: boil variation 33
Task Description: Your task is to use chemistry to create violet-red paint. When you are part-way done, focus on the intermediate (secondary color) paint you created.  When you are completely done, focus on the violet paint.


'use chemistry to create violet-red paint'

In [22]:
def step_function(action: str) -> State:
    observation, reward, isCompleted, info = env.step(action)
    updated_state = parse_observation(observation=observation,
                                      inventory=info['inv'],
                                      look_around=info['look'],
                                      task=goal,
                                      valid_actions=info['valid'],
                                      score=info['score'])
    return updated_state


agent = BDIAgent(plan_library=pl, nli_model=nli_model)
last_state = agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())
print(last_state.reward)

[(0.6918235421180725, Plan(triggering_event='use chemistry to create violet-red paint', context=['you are not in art studio'], body=['move to art studio', 'pour cup containing red paint in art studio into cup containing nothing', 'pour cup containing blue paint in art studio into cup containing red paint in table', 'mix cup containing blue paint and red paint', 'focus on paint in cup containing violet paint', 'pour cup containing violet paint into cup containing violet paint', 'open cupboard', 'pour cup containing red paint into cup containing violet paint', 'mix cup containing red paint and violet paint', 'focus on paint in cup containing violet-red paint'], idx=1))]
Plan Selected Plan(triggering_event='use chemistry to create violet-red paint', context=['you are not in art studio'], body=['move to art studio', 'pour cup containing red paint in art studio into cup containing nothing', 'pour cup containing blue paint in art studio into cup containing red paint in table', 'mix cup conta

In [23]:
len(agent.plan_library.plans[goal])

2

In [24]:
[a for a in last_state.valid_actions if a.startswith("pour cup containing red paint in art studio ")]

[]

In [25]:
[a for a in last_state.valid_actions if a.startswith("open")]

['open drawer']

In [26]:

observation, reward, isCompleted, info = env.step('open art studio door')
observation

'No known action matches that input.'

In [27]:
info['look']

'This room is called the art studio. In it, you see: \n\tthe agent\n\ta substance called air\n\ta large cupboard. The large cupboard door is open. In the large cupboard is: a drawer, a wood cup (containing yellow paint), a wood cup (containing blue paint), a wood cup (containing nothing).\n\ta table. On the table is: a glass cup (containing violet-red paint).\n\ta wood cup (containing nothing)\n\ta wood cup (containing nothing)\n\ta wood cup (containing yellow paint)\nYou also see:\n\tA door to the hallway (that is open)\n'